## BERT

In this notebook, we will use a wrapper library called ktrain to train our model using the pre-trained language model by Google- BERT. We will be using a compressed version of the model to facilitate training called Distilbert


In [1]:
!pip3 install -q tensorflow_gpu==2.1.0

     |████████████████████████████████| 421.8MB 31kB/s 
     |████████████████████████████████| 3.9MB 40.0MB/s 
     |████████████████████████████████| 450kB 48.9MB/s 
ERROR: tensorflow 2.2.0rc3 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [3]:
!pip3 install -q ktrain

     |████████████████████████████████| 25.2MB 66.8MB/s 
     |████████████████████████████████| 421.8MB 23kB/s 
     |████████████████████████████████| 983kB 44.8MB/s 
     |████████████████████████████████| 245kB 54.8MB/s 
     |████████████████████████████████| 573kB 49.8MB/s 
     |████████████████████████████████| 471kB 48.1MB/s 
     |████████████████████████████████| 1.0MB 45.3MB/s 
     |████████████████████████████████| 3.7MB 45.8MB/s 
     |████████████████████████████████| 890kB 55.9MB/s 


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
df = pd.read_csv('drive/My Drive/dataset_final.csv', engine='python')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
sr = pd.isnull(df['title'])
sr.loc[sr==True]
df.drop(83369, axis=0, inplace=True)
df = df.reset_index(drop=True)
sr = pd.isnull(df['full_text'])
drop_arr = sr.loc[sr==True].index.tolist()
df.drop(drop_arr, axis=0, inplace=True)
df = df.reset_index(drop=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106022 entries, 0 to 106021
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   title         106022 non-null  object 
 1   flair         106022 non-null  object 
 2   score         106022 non-null  float64
 3   num_comments  106022 non-null  float64
 4   author        106022 non-null  object 
 5   created_utc   106022 non-null  float64
 6   self_post     106022 non-null  object 
 7   over_18       106022 non-null  object 
 8   full_text     106022 non-null  object 
dtypes: float64(3), object(6)
memory usage: 7.3+ MB


In [0]:
target_flairs = df['flair'].index.tolist()
X_text = list(df['full_text'])
Y = list(df['flair'])
x_train, x_test, y_train, y_test = train_test_split(X_text, Y, test_size=0.3)

In [9]:
import ktrain
from ktrain import text
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, class_names=target_flairs)
trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)


preprocessing train...
language: en
train sequence lengths:
	mean : 35
	95percentile : 142
	99percentile : 403


/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:478: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


preprocessing test...
language: en
test sequence lengths:
	mean : 35
	95percentile : 140
	99percentile : 424


In [0]:
learner.fit_onecycle(5e-5, 3)
predictor = ktrain.get_predictor(learner.model, preproc=t)
predictor.save('drive/My Drive/distilbert_predictor_final')
learner.validate(class_names=t.get_classes())



begin training using onecycle policy with max lr of 5e-05...
Train for 12370 steps, validate for 994 steps
Epoch 1/3
12370/12370 [==============================] - 6770s 547ms/step - loss: 1.0872 - accuracy: 0.6289 - val_loss: 0.9532 - val_accuracy: 0.6671
Epoch 2/3
12370/12370 [==============================] - 6744s 545ms/step - loss: 0.9169 - accuracy: 0.6815 - val_loss: 0.9449 - val_accuracy: 0.6683
Epoch 3/3
12370/12370 [==============================] - 6746s 545ms/step - loss: 0.6600 - accuracy: 0.7739 - val_loss: 0.9984 - val_accuracy: 0.6688
                    precision    recall  f1-score   support

          AskIndia       0.72      0.80      0.76      3810
  Business/Finance       0.47      0.41      0.44      1385
       Coronavirus       0.59      0.64      0.61       280
     Non-Political       0.66      0.68      0.67     10242
       Photography       0.66      0.41      0.50       328
    Policy/Economy       0.57      0.60      0.59      3894
          Politics  

array([[3034,   24,   15,  407,    5,   82,  183,   38,    2,   20],
       [  74,  564,    0,  312,    2,  273,   48,  106,    6,    0],
       [  11,    1,  179,   54,    0,    3,   28,    4,    0,    0],
       [ 629,  278,   69, 6934,   51,  601, 1288,  293,   69,   30],
       [   5,    1,    0,  173,  133,    0,   12,    3,    1,    0],
       [  92,  150,   10,  546,    3, 2339,  647,   89,    9,    9],
       [ 124,   56,   20, 1182,    3,  652, 6930,   64,   20,   24],
       [  69,  104,   10,  386,    3,   98,   50,  635,    3,    2],
       [   3,    3,    0,   80,    0,    5,   27,    1,  348,    1],
       [ 163,   11,    0,  368,    1,   36,  181,   25,    4,  176]])